In [6]:
# 데이터 증강
# Resize / Rotation / RandomErasing

import os, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import cv2

import torchvision.transforms as transforms
from PIL import Image

# 디바이스 설정
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

# 하이퍼파라미터 설정
slice_root = "/data1/lidc-idri/slices"
batch_size = 16
num_epochs = 100
learning_rate = 1e-4


# --- Transform 정의 ---
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),  # 먼저 Tensor로 변환
    transforms.Normalize([0.5], [0.5]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.1), ratio=(0.3, 3.3), value=0)  # 마지막에 적용
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])



# -------------------- 데이터 --------------------
def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None if score == 3 else int(score >= 4)
    except: return None

class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        img = np.load(self.paths[idx])
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.  # 정규화 (0~1)
        img = np.expand_dims(img, axis=-1)  # (H, W, 1)
        if self.transform:
            img = self.transform(img)
        else:
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)  # [C, H, W]
        return img, torch.tensor(self.labels[idx]).long()

    def __len__(self): return len(self.paths)


# -------------------- CBAM & BAM --------------------
class ChannelAttention(nn.Module):
    def __init__(self, planes, ratio=16):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False), nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        self.avg, self.max, self.sigmoid = nn.AdaptiveAvgPool2d(1), nn.AdaptiveMaxPool2d(1), nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))

class SpatialAttention(nn.Module):
    def __init__(self, k=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg, _max = torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))

class CBAM(nn.Module):
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

    def forward(self, x):
        return self.sa(self.ca(x) * x) * x
    



# -------------------- ResNet + CBAM --------------------
class BasicBlockCBAM(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()

        self.conv1 = nn.Conv2d(in_planes, out_planes, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU()

        self.conv2 = nn.Conv2d(out_planes, out_planes, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

        self.cbam = CBAM(out_planes) if use_cbam else None
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.cbam: out = self.cbam(out)
        if self.downsample: residual = self.downsample(x)
        return self.relu(out + residual)

class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, 7, 2, 3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(3, 2, 1)

        self.layer1 = self._make_layer(64, 2)                 # CBAM 적용
        self.layer2 = self._make_layer(128, 2, stride=2)       # CBAM 적용
        self.layer3 = self._make_layer(256, 2, stride=2)       # CBAM 적용
        self.layer4 = self._make_layer(512, 2, stride=2, use_cbam=False)  # CBAM 미적용

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, planes, blocks, stride=1, use_cbam=True):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        layers = [BasicBlockCBAM(self.in_planes, planes, stride, downsample, use_cbam=use_cbam)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlockCBAM(self.in_planes, planes, use_cbam=use_cbam))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.maxpool(self.relu(self.bn1(self.conv1(x))))
        x = self.layer4(self.layer3(self.layer2(self.layer1(x))))
        x = self.avgpool(x)
        return self.fc(torch.flatten(x, 1))
    



# -------------------- 학습 & 평가 --------------------
def run():
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)
    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_loader = DataLoader(CTDataset(train_files, train_labels, transform=train_transform), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(CTDataset(val_files, val_labels, transform=val_transform), batch_size=batch_size)
    test_loader = DataLoader(CTDataset(test_files, test_labels, transform=val_transform), batch_size=batch_size)

    model = ResNet18_CBAM().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc, save_path = 0.0, "aug_resnet_cbam.pth"

    for epoch in range(num_epochs):
        model.train(); epoch_loss = 0; correct = total = 0
        for imgs, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward(); optimizer.step()
            epoch_loss += loss.item()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")

        # Validation
        model.eval(); correct = total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # --- 테스트 ---
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path)); model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    print(f"✅ Test Accuracy: {(np.array(y_pred) == np.array(y_true)).mean() * 100:.2f}%")
    print(classification_report(y_true, y_pred, digits=4))
    print(f"AUC: {roc_auc_score(y_true, y_probs):.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

if __name__ == "__main__":
    run()

cuda:1


[Epoch 1]: 100%|██████████| 234/234 [00:15<00:00, 14.63it/s]


Train Acc: 64.8714, Loss: 0.6455
Val Acc: 0.6937
✅ Saved best model!


[Epoch 2]: 100%|██████████| 234/234 [00:14<00:00, 15.97it/s]


Train Acc: 66.6935, Loss: 0.6039
Val Acc: 0.7175
✅ Saved best model!


...

Train Acc: 98.3655, Loss: 0.0482
Val Acc: 0.8950


[Epoch 99]: 100%|██████████| 234/234 [00:15<00:00, 15.13it/s]


Train Acc: 97.7224, Loss: 0.0621
Val Acc: 0.8875


[Epoch 100]: 100%|██████████| 234/234 [00:17<00:00, 13.28it/s]


Train Acc: 97.9368, Loss: 0.0575
Val Acc: 0.8850

📊 Test Evaluation:
✅ Test Accuracy: 89.12%
              precision    recall  f1-score   support

           0     0.8534    0.8255    0.8392       275
           1     0.9101    0.9257    0.9178       525

    accuracy                         0.8912       800
   macro avg     0.8817    0.8756    0.8785       800
weighted avg     0.8906    0.8912    0.8908       800

AUC: 0.9256
Confusion Matrix:
[[227  48]
 [ 39 486]]


In [ ]:
# 데이터 증강
# Rotation /  / 

import os, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import cv2

import torchvision.transforms as transforms
from PIL import Image

# 디바이스 설정
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

# 하이퍼파라미터 설정
slice_root = "/data1/lidc-idri/slices"
batch_size = 16
num_epochs = 100
learning_rate = 1e-4


# --- Transform 정의 ---
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    # transforms.CenterCrop(180),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    # transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    # transforms.CenterCrop(180),
    transforms.ToTensor()
])



# -------------------- 데이터 --------------------
def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None if score == 3 else int(score >= 4)
    except: return None

class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        img = np.load(self.paths[idx])
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.  # 정규화 (0~1)
        img = np.expand_dims(img, axis=-1)  # (H, W, 1)
        if self.transform:
            img = self.transform(img)
        else:
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)  # [C, H, W]
        return img, torch.tensor(self.labels[idx]).long()

    def __len__(self): return len(self.paths)


# -------------------- CBAM & BAM --------------------
class ChannelAttention(nn.Module):
    def __init__(self, planes, ratio=16):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False), nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        self.avg, self.max, self.sigmoid = nn.AdaptiveAvgPool2d(1), nn.AdaptiveMaxPool2d(1), nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))

class SpatialAttention(nn.Module):
    def __init__(self, k=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg, _max = torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))

class CBAM(nn.Module):
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

    def forward(self, x):
        return self.sa(self.ca(x) * x) * x
    



# -------------------- ResNet + CBAM --------------------
class BasicBlockCBAM(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()

        self.conv1 = nn.Conv2d(in_planes, out_planes, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU()

        self.conv2 = nn.Conv2d(out_planes, out_planes, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

        self.cbam = CBAM(out_planes) if use_cbam else None
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.cbam: out = self.cbam(out)
        if self.downsample: residual = self.downsample(x)
        return self.relu(out + residual)

class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, 7, 2, 3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(3, 2, 1)

        self.layer1 = self._make_layer(64, 2)                 # CBAM 적용
        self.layer2 = self._make_layer(128, 2, stride=2)       # CBAM 적용
        self.layer3 = self._make_layer(256, 2, stride=2)       # CBAM 적용
        self.layer4 = self._make_layer(512, 2, stride=2, use_cbam=False)  # CBAM 미적용

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, planes, blocks, stride=1, use_cbam=True):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        layers = [BasicBlockCBAM(self.in_planes, planes, stride, downsample, use_cbam=use_cbam)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlockCBAM(self.in_planes, planes, use_cbam=use_cbam))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.maxpool(self.relu(self.bn1(self.conv1(x))))
        x = self.layer4(self.layer3(self.layer2(self.layer1(x))))
        x = self.avgpool(x)
        return self.fc(torch.flatten(x, 1))
    



# -------------------- 학습 & 평가 --------------------
def run():
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)
    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_loader = DataLoader(CTDataset(train_files, train_labels, transform=train_transform), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(CTDataset(val_files, val_labels, transform=val_transform), batch_size=batch_size)
    test_loader = DataLoader(CTDataset(test_files, test_labels, transform=val_transform), batch_size=batch_size)

    model = ResNet18_CBAM().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc, save_path = 0.0, "aug_resnet_cbam.pth"

    for epoch in range(num_epochs):
        model.train(); epoch_loss = 0; correct = total = 0
        for imgs, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward(); optimizer.step()
            epoch_loss += loss.item()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")

        # Validation
        model.eval(); correct = total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # --- 테스트 ---
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path)); model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    print(f"✅ Test Accuracy: {(np.array(y_pred) == np.array(y_true)).mean() * 100:.2f}%")
    print(classification_report(y_true, y_pred, digits=4))
    print(f"AUC: {roc_auc_score(y_true, y_probs):.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

if __name__ == "__main__":
    run()

cuda:1


[Epoch 1]: 100%|██████████| 234/234 [00:07<00:00, 29.44it/s]


Train Acc: 64.3355, Loss: 0.6515
Val Acc: 0.6850
✅ Saved best model!


[Epoch 2]: 100%|██████████| 234/234 [00:07<00:00, 29.46it/s]


Train Acc: 65.3269, Loss: 0.6360
Val Acc: 0.6600


...

Train Acc: 94.5338, Loss: 0.1359
Val Acc: 0.8462


[Epoch 99]: 100%|██████████| 234/234 [00:13<00:00, 17.22it/s]


Train Acc: 95.6056, Loss: 0.1114
Val Acc: 0.8063


[Epoch 100]: 100%|██████████| 234/234 [00:14<00:00, 16.51it/s]


Train Acc: 94.9893, Loss: 0.1308
Val Acc: 0.8525

📊 Test Evaluation:
✅ Test Accuracy: 85.50%
              precision    recall  f1-score   support

           0     0.7955    0.7782    0.7868       275
           1     0.8851    0.8952    0.8902       525

    accuracy                         0.8550       800
   macro avg     0.8403    0.8367    0.8385       800
weighted avg     0.8543    0.8550    0.8546       800

AUC: 0.9080
Confusion Matrix:
[[214  61]
 [ 55 470]]


In [ ]:
# 데이터 증강
# Rotation / CenterCrop / GaussianBlur

import os, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import cv2

import torchvision.transforms as transforms
from PIL import Image

# 디바이스 설정
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

# 하이퍼파라미터 설정
slice_root = "/data1/lidc-idri/slices"
batch_size = 16
num_epochs = 100
learning_rate = 1e-4


# --- Transform 정의 ---
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.CenterCrop(180),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.CenterCrop(180),
    transforms.ToTensor()
])



# -------------------- 데이터 --------------------
def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None if score == 3 else int(score >= 4)
    except: return None

class CTDataset(Dataset):
    def __init__(self, paths, labels, transform=None):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        img = np.load(self.paths[idx])
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.  # 정규화 (0~1)
        img = np.expand_dims(img, axis=-1)  # (H, W, 1)
        if self.transform:
            img = self.transform(img)
        else:
            img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)  # [C, H, W]
        return img, torch.tensor(self.labels[idx]).long()

    def __len__(self): return len(self.paths)


# -------------------- CBAM & BAM --------------------
class ChannelAttention(nn.Module):
    def __init__(self, planes, ratio=16):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False), nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        self.avg, self.max, self.sigmoid = nn.AdaptiveAvgPool2d(1), nn.AdaptiveMaxPool2d(1), nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))

class SpatialAttention(nn.Module):
    def __init__(self, k=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg, _max = torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))

class CBAM(nn.Module):
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

    def forward(self, x):
        return self.sa(self.ca(x) * x) * x
    



# -------------------- ResNet + CBAM --------------------
class BasicBlockCBAM(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()

        self.conv1 = nn.Conv2d(in_planes, out_planes, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU()

        self.conv2 = nn.Conv2d(out_planes, out_planes, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

        self.cbam = CBAM(out_planes) if use_cbam else None
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.cbam: out = self.cbam(out)
        if self.downsample: residual = self.downsample(x)
        return self.relu(out + residual)

class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, 7, 2, 3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(3, 2, 1)

        self.layer1 = self._make_layer(64, 2)                 # CBAM 적용
        self.layer2 = self._make_layer(128, 2, stride=2)       # CBAM 적용
        self.layer3 = self._make_layer(256, 2, stride=2)       # CBAM 적용
        self.layer4 = self._make_layer(512, 2, stride=2, use_cbam=False)  # CBAM 미적용

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, planes, blocks, stride=1, use_cbam=True):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        layers = [BasicBlockCBAM(self.in_planes, planes, stride, downsample, use_cbam=use_cbam)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlockCBAM(self.in_planes, planes, use_cbam=use_cbam))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.maxpool(self.relu(self.bn1(self.conv1(x))))
        x = self.layer4(self.layer3(self.layer2(self.layer1(x))))
        x = self.avgpool(x)
        return self.fc(torch.flatten(x, 1))
    



# -------------------- 학습 & 평가 --------------------
def run():
    all_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)
    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_loader = DataLoader(CTDataset(train_files, train_labels, transform=train_transform), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(CTDataset(val_files, val_labels, transform=val_transform), batch_size=batch_size)
    test_loader = DataLoader(CTDataset(test_files, test_labels, transform=val_transform), batch_size=batch_size)

    model = ResNet18_CBAM().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_acc, save_path = 0.0, "aug_resnet_cbam.pth"

    for epoch in range(num_epochs):
        model.train(); epoch_loss = 0; correct = total = 0
        for imgs, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward(); optimizer.step()
            epoch_loss += loss.item()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        print(f"Train Acc: {(correct/total)*100:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")

        # Validation
        model.eval(); correct = total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # --- 테스트 ---
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path)); model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    print(f"✅ Test Accuracy: {(np.array(y_pred) == np.array(y_true)).mean() * 100:.2f}%")
    print(classification_report(y_true, y_pred, digits=4))
    print(f"AUC: {roc_auc_score(y_true, y_probs):.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

if __name__ == "__main__":
    run()

cuda:1


[Epoch 1]: 100%|██████████| 234/234 [00:06<00:00, 38.33it/s]


Train Acc: 64.1479, Loss: 0.6555
Val Acc: 0.6175
✅ Saved best model!


[Epoch 2]: 100%|██████████| 234/234 [00:06<00:00, 37.37it/s]


Train Acc: 65.8360, Loss: 0.6293
Val Acc: 0.6625
✅ Saved best model!


...

Train Acc: 98.5798, Loss: 0.0357
Val Acc: 0.8300


[Epoch 99]: 100%|██████████| 234/234 [00:05<00:00, 39.18it/s]


Train Acc: 98.2047, Loss: 0.0604
Val Acc: 0.8550


[Epoch 100]: 100%|██████████| 234/234 [00:06<00:00, 38.28it/s]


Train Acc: 98.2583, Loss: 0.0545
Val Acc: 0.8425

📊 Test Evaluation:
✅ Test Accuracy: 83.88%
              precision    recall  f1-score   support

           0     0.7786    0.7418    0.7598       275
           1     0.8680    0.8895    0.8786       525

    accuracy                         0.8387       800
   macro avg     0.8233    0.8157    0.8192       800
weighted avg     0.8373    0.8387    0.8378       800

AUC: 0.8805
Confusion Matrix:
[[204  71]
 [ 58 467]]


In [1]:
# layer4 에 CBAM 안들어간 ver

import os, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from glob import glob
from tqdm import tqdm
import cv2

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 100
LR = 1e-4

# -------------------- 데이터 --------------------
def extract_label_from_filename(fname):
    try:
        score = int(fname.split("_")[-1].replace(".npy", ""))
        return None if score == 3 else int(score >= 4)
    except: return None

class CTDataset(Dataset):
    def __init__(self, paths, labels):
        self.paths = paths
        self.labels = labels

    def __getitem__(self, idx):
        img = np.load(self.paths[idx])
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.
        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, 0).astype(np.float32)
        return torch.tensor(img), torch.tensor(self.labels[idx]).long()

    def __len__(self): return len(self.paths)


    

# -------------------- CBAM & BAM --------------------
class ChannelAttention(nn.Module):
    def __init__(self, planes, ratio=16):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Conv2d(planes, planes // ratio, 1, bias=False), nn.ReLU(),
            nn.Conv2d(planes // ratio, planes, 1, bias=False))
        self.avg, self.max, self.sigmoid = nn.AdaptiveAvgPool2d(1), nn.AdaptiveMaxPool2d(1), nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.shared(self.avg(x)) + self.shared(self.max(x)))

class SpatialAttention(nn.Module):
    def __init__(self, k=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=k, padding=k // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg, _max = torch.mean(x, dim=1, keepdim=True), torch.max(x, dim=1, keepdim=True)[0]
        return self.sigmoid(self.conv(torch.cat([avg, _max], dim=1)))

class CBAM(nn.Module):
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

    def forward(self, x):
        return self.sa(self.ca(x) * x) * x
    



# -------------------- ResNet + CBAM --------------------
class BasicBlockCBAM(nn.Module):
    def __init__(self, in_planes, out_planes, stride=1, downsample=None, use_cbam=True):
        super().__init__()

        self.conv1 = nn.Conv2d(in_planes, out_planes, 3, stride, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_planes)
        self.relu = nn.ReLU()

        self.conv2 = nn.Conv2d(out_planes, out_planes, 3, 1, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)

        self.cbam = CBAM(out_planes) if use_cbam else None
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.cbam: out = self.cbam(out)
        if self.downsample: residual = self.downsample(x)
        return self.relu(out + residual)

class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, 7, 2, 3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(3, 2, 1)

        self.layer1 = self._make_layer(64, 2)                 # CBAM 적용
        self.layer2 = self._make_layer(128, 2, stride=2)       # CBAM 적용
        self.layer3 = self._make_layer(256, 2, stride=2)       # CBAM 적용
        self.layer4 = self._make_layer(512, 2, stride=2, use_cbam=False)  # CBAM 미적용

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, planes, blocks, stride=1, use_cbam=True):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, 1, stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        layers = [BasicBlockCBAM(self.in_planes, planes, stride, downsample, use_cbam=use_cbam)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlockCBAM(self.in_planes, planes, use_cbam=use_cbam))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.maxpool(self.relu(self.bn1(self.conv1(x))))
        x = self.layer4(self.layer3(self.layer2(self.layer1(x))))
        x = self.avgpool(x)
        return self.fc(torch.flatten(x, 1))
    



# -------------------- 학습 & 평가 --------------------
def run():
    all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
    file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
    file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
    files, labels = zip(*file_label_pairs)
    train_files, temp_files, train_labels, temp_labels = train_test_split(files, labels, test_size=0.3, random_state=42)
    val_files, test_files, val_labels, test_labels = train_test_split(temp_files, temp_labels, test_size=0.5, random_state=42)

    train_loader = DataLoader(CTDataset(train_files, train_labels), batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(CTDataset(val_files, val_labels), batch_size=BATCH_SIZE)
    test_loader = DataLoader(CTDataset(test_files, test_labels), batch_size=BATCH_SIZE)

    model = ResNet18_CBAM().to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    best_acc, save_path = 0.0, "best_resnet_cbam.pth"

    for epoch in range(NUM_EPOCHS):
        model.train(); epoch_loss = 0; correct = total = 0
        for imgs, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward(); optimizer.step()
            epoch_loss += loss.item()
            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
        print(f"Train Acc: {correct/total:.4f}, Loss: {epoch_loss/len(train_loader):.4f}")

        # Validation
        model.eval(); correct = total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
                outputs = model(imgs)
                _, preds = outputs.max(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct / total
        print(f"Val Acc: {val_acc:.4f}")
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), save_path)
            print("✅ Saved best model!")

    # --- 테스트 ---
    print("\n📊 Test Evaluation:")
    model.load_state_dict(torch.load(save_path)); model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)
            probs = F.softmax(outputs, dim=1)[:, 1]
            preds = outputs.argmax(1)
            y_probs.extend(probs.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            y_true.extend(labels.cpu().numpy())
    print(f"✅ Test Accuracy: {(np.array(y_pred) == np.array(y_true)).mean() * 100:.2f}%")
    print(classification_report(y_true, y_pred, digits=4))
    print(f"AUC: {roc_auc_score(y_true, y_probs):.4f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))

if __name__ == "__main__":
    run()

[Epoch 1]: 100%|██████████| 234/234 [00:15<00:00, 15.48it/s]


Train Acc: 0.6881, Loss: 0.6066
Val Acc: 0.5537
✅ Saved best model!


[Epoch 2]: 100%|██████████| 234/234 [00:15<00:00, 15.31it/s]


Train Acc: 0.8130, Loss: 0.4289
Val Acc: 0.7900
✅ Saved best model!


[Epoch 3]: 100%|██████████| 234/234 [00:16<00:00, 14.36it/s]


...

Train Acc: 0.9984, Loss: 0.0020
Val Acc: 0.8750


[Epoch 99]: 100%|██████████| 234/234 [00:15<00:00, 15.48it/s]


Train Acc: 0.9984, Loss: 0.0020
Val Acc: 0.8812


[Epoch 100]: 100%|██████████| 234/234 [00:17<00:00, 13.48it/s]


Train Acc: 0.9579, Loss: 0.1089
Val Acc: 0.8700

📊 Test Evaluation:
✅ Test Accuracy: 89.50%
              precision    recall  f1-score   support

           0     0.8659    0.8218    0.8433       275
           1     0.9091    0.9333    0.9211       525

    accuracy                         0.8950       800
   macro avg     0.8875    0.8776    0.8822       800
weighted avg     0.8942    0.8950    0.8943       800

AUC: 0.9307
Confusion Matrix:
[[226  49]
 [ 35 490]]


In [ ]:
# layer4 에 CBAM 들어간 ver

import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

# 설정
SLICE_ROOT = "/data1/lidc-idri/slices"
BATCH_SIZE = 16
NUM_EPOCHS = 100
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# 레이블 추출
def extract_label_from_filename(filename):
    try:
        score = int(filename.split("_")[-1].replace(".npy", ""))
        if score == 3: return None
        return 1 if score >= 4 else 0
    except:
        return None

# 파일 분할
all_files = glob(os.path.join(SLICE_ROOT, "LIDC-IDRI-*", "*.npy"))
file_label_pairs = [(f, extract_label_from_filename(f)) for f in all_files]
file_label_pairs = [(f, l) for f, l in file_label_pairs if l is not None]
files, labels = zip(*file_label_pairs)

train_files, temp_files, train_labels, temp_labels = train_test_split(
    files, labels, test_size=0.3, random_state=42)
val_files, test_files, val_labels, test_labels = train_test_split(
    temp_files, temp_labels, test_size=0.5, random_state=42)

# Dataset 클래스
class CTBinaryClassificationDataset(Dataset):
    def __init__(self, file_list, label_list):
        self.files = file_list
        self.labels = label_list

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = np.load(self.files[idx])
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.0
        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, axis=0).astype(np.float32)
        label = self.labels[idx]
        return torch.tensor(img), torch.tensor(label).long()

# CBAM + ResNet18 정의 (생략하지 않고 유지)
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super().__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.shared_mlp = nn.Sequential(
            nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False),
            nn.ReLU(),
            nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.shared_mlp(self.avg_pool(x))
        max_out = self.shared_mlp(self.max_pool(x))
        return self.sigmoid(avg_out + max_out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        return self.sigmoid(self.conv(x_cat))

class CBAM(nn.Module):
    def __init__(self, planes):
        super().__init__()
        self.ca = ChannelAttention(planes)
        self.sa = SpatialAttention()

    def forward(self, x):
        x = self.ca(x) * x
        x = self.sa(x) * x
        return x

class BasicBlockWithCBAM(nn.Module):
    def __init__(self, in_planes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.cbam = CBAM(planes)

        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.cbam(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class ResNet18_CBAM(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(64, 2)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_planes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )
        layers = [BasicBlockWithCBAM(self.in_planes, planes, stride, downsample)]
        self.in_planes = planes
        for _ in range(1, blocks):
            layers.append(BasicBlockWithCBAM(self.in_planes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)

# 데이터로더
train_loader = DataLoader(CTBinaryClassificationDataset(train_files, train_labels), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(CTBinaryClassificationDataset(val_files, val_labels), batch_size=BATCH_SIZE)
test_loader = DataLoader(CTBinaryClassificationDataset(test_files, test_labels), batch_size=BATCH_SIZE)

# 모델 및 학습 설정
model = ResNet18_CBAM(num_classes=2).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

save_dir = os.path.join(os.path.dirname(os.getcwd()), "pth")
os.makedirs(save_dir, exist_ok=True)
best_val_acc = 0.0

# 학습
for epoch in range(NUM_EPOCHS):
    model.train()
    correct = total = epoch_loss = 0
    for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}]"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    train_acc = correct / total * 100
    print(f"Train Loss: {epoch_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%")

    # 검증
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    val_acc = correct / total * 100
    print(f"Validation Accuracy: {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), os.path.join(save_dir, "best_model_resnet18_CBAM.pth"))
        print("✅ Best model saved!")


# 최종 테스트
print("\n📊 Test Set Evaluation (Best Model 기준):")
model.load_state_dict(torch.load(os.path.join(save_dir, "best_model_resnet18_CBAM.pth")))
model.eval()
y_true, y_pred, y_probs = [], [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        probs = F.softmax(outputs, dim=1)[:, 1]  # malignant 확률
        preds = torch.argmax(outputs, dim=1)

        y_probs.extend(probs.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# Accuracy 출력
acc = (np.array(y_pred) == np.array(y_true)).mean() * 100
print(f"✅ Test Accuracy: {acc:.2f}%")

# Classification Report
print(classification_report(y_true, y_pred, digits=4))

# AUC 출력
try:
    
    auc_score = roc_auc_score(y_true, y_probs)
    print(f"AUC: {auc_score:.4f}")
except ValueError:
    print("AUC 계산 실패: 양/음 클래스가 모두 있어야 함.")

# Confusion Matrix 출력
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)
